# Estatísticas em PySpark



## 0. Importações e Sessão Spark

In [1]:
# Instalar o OpenJDK 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixar o Apache Spark (versão 3.5.0 com Hadoop 3)
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

# Extrair o arquivo baixado
!tar xf spark-3.5.0-bin-hadoop3.tgz

# Instalar a biblioteca findspark
!pip install -q findspark

import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import Window

print("Iniciando sessão Spark...")
spark = (SparkSession.builder
                    .appName('EstatisticasPySpark')
                    .getOrCreate())
print("Sessão Spark criada com sucesso!")

🚀 Iniciando sessão Spark...
✅ Sessão Spark criada com sucesso!


## 1. Criando um Dataset de Exemplo

In [3]:
schema = StructType([
    StructField("nome", StringType(), True),
    StructField("idade", IntegerType(), True),
    StructField("cidade", StringType(), True),
    StructField("salario", DoubleType(), True)
])

data = [
    ("Alice", 25, "São Paulo", 5000.0),
    ("Bruno", 30, "Rio de Janeiro", 6500.0),
    ("Carlos", None, "Belo Horizonte", 4800.0),
    ("Diana", 35, "Porto Alegre", 7200.0),
    ("Eduardo", 28, None, 5500.0),
    ("Fernanda", 32, "Salvador", None),
    ("Gabriel", 29, "Brasília", 6800.0),
    ("Helena", 27, "Recife", 5300.0),
    ("Igor", 31, "Fortaleza", 6000.0),
    ("Juliana", 26, "Curitiba", 5800.0)
]

df = spark.createDataFrame(data, schema)
df.show()
print(f"📈 Dimensões: {df.count()} linhas, {len(df.columns)} colunas")
print(f"📋 Colunas: {df.columns}")

+--------+-----+--------------+-------+
|    nome|idade|        cidade|salario|
+--------+-----+--------------+-------+
|   Alice|   25|     São Paulo| 5000.0|
|   Bruno|   30|Rio de Janeiro| 6500.0|
|  Carlos| NULL|Belo Horizonte| 4800.0|
|   Diana|   35|  Porto Alegre| 7200.0|
| Eduardo|   28|          NULL| 5500.0|
|Fernanda|   32|      Salvador|   NULL|
| Gabriel|   29|      Brasília| 6800.0|
|  Helena|   27|        Recife| 5300.0|
|    Igor|   31|     Fortaleza| 6000.0|
| Juliana|   26|      Curitiba| 5800.0|
+--------+-----+--------------+-------+

📈 Dimensões: 10 linhas, 4 colunas
📋 Colunas: ['nome', 'idade', 'cidade', 'salario']


## 2. Estatísticas Básicas com `describe()`

In [4]:
df.describe().show()
df.describe(['idade']).show()
df.describe(['idade', 'salario']).show()

+-------+-------+------------------+--------------+-----------------+
|summary|   nome|             idade|        cidade|          salario|
+-------+-------+------------------+--------------+-----------------+
|  count|     10|                 9|             9|                9|
|   mean|   NULL| 29.22222222222222|          NULL|5877.777777777777|
| stddev|   NULL|3.1534813214040835|          NULL|822.7663364798323|
|    min|  Alice|                25|Belo Horizonte|           4800.0|
|    max|Juliana|                35|     São Paulo|           7200.0|
+-------+-------+------------------+--------------+-----------------+

+-------+------------------+
|summary|             idade|
+-------+------------------+
|  count|                 9|
|   mean| 29.22222222222222|
| stddev|3.1534813214040835|
|    min|                25|
|    max|                35|
+-------+------------------+

+-------+------------------+-----------------+
|summary|             idade|          salario|
+-------+----

## 3. Estatísticas Avançadas com `summary()`

In [5]:
df.summary().show()
df.summary('mean').show()
df.summary('25%', '50%', '75%', '90%').show()
df.summary('count', 'mean', 'stddev').show()

+-------+-------+------------------+--------------+-----------------+
|summary|   nome|             idade|        cidade|          salario|
+-------+-------+------------------+--------------+-----------------+
|  count|     10|                 9|             9|                9|
|   mean|   NULL| 29.22222222222222|          NULL|5877.777777777777|
| stddev|   NULL|3.1534813214040835|          NULL|822.7663364798323|
|    min|  Alice|                25|Belo Horizonte|           4800.0|
|    25%|   NULL|                27|          NULL|           5300.0|
|    50%|   NULL|                29|          NULL|           5800.0|
|    75%|   NULL|                31|          NULL|           6500.0|
|    max|Juliana|                35|     São Paulo|           7200.0|
+-------+-------+------------------+--------------+-----------------+

+-------+----+-----------------+------+-----------------+
|summary|nome|            idade|cidade|          salario|
+-------+----+-----------------+------+----

## 4. Estatísticas Individuais com Funções do PySpark

In [6]:
estatisticas_salario = df.agg(
    F.count('salario').alias('total_registros'),
    F.countDistinct('salario').alias('valores_unicos'),
    F.mean('salario').alias('media'),
    F.median('salario').alias('mediana'),
    F.stddev('salario').alias('desvio_padrao'),
    F.variance('salario').alias('variancia'),
    F.min('salario').alias('minimo'),
    F.max('salario').alias('maximo'),
    F.sum('salario').alias('soma_total')
).collect()[0]

estatisticas_salario

Row(total_registros=9, valores_unicos=9, media=5877.777777777777, mediana=5800.0, desvio_padrao=822.7663364798323, variancia=676944.4444444445, minimo=4800.0, maximo=7200.0, soma_total=52900.0)

## 5. Geração de Números Aleatórios

O PySpark oferece a função rand() para gerar números aleatórios uniformemente distribuídos entre 0.0 e 1.0. Essa função é muito útil para simulações, amostragem de dados, criação de datasets sintéticos ou testes.

- Adiciona ao DataFrame df uma nova coluna chamada numero_aleatorio contendo números aleatórios entre 0 e 1.
- Adiciona uma coluna de números aleatórios com uma seed fixa (seed=42).
- Multiplicação e conversão permitem criar números em diferentes intervalos e tipos (float ou inteiro).

In [7]:
df_com_random = df.withColumn('numero_aleatorio', F.rand())
df_com_random.select('nome', 'numero_aleatorio').show(5)

df_com_seed = df.withColumn('random_reproduzivel', F.rand(seed=42))
df_com_seed.select('nome', 'random_reproduzivel').show(5)

df_random_completo = df.withColumn('aleatorio_0_1', F.rand(seed=123)) \
                       .withColumn('aleatorio_1_100', F.rand(seed=123) * 100) \
                       .withColumn('aleatorio_inteiro', (F.rand(seed=123) * 100).cast('int'))
df_random_completo.select('nome', 'aleatorio_0_1', 'aleatorio_1_100', 'aleatorio_inteiro').show(5)

+-------+------------------+
|   nome|  numero_aleatorio|
+-------+------------------+
|  Alice|0.2295271979881215|
|  Bruno|0.6721950494773951|
| Carlos|0.5139176584798829|
|  Diana|0.3014148634612559|
|Eduardo| 0.900838478657975|
+-------+------------------+
only showing top 5 rows

+-------+-------------------+
|   nome|random_reproduzivel|
+-------+-------------------+
|  Alice|  0.619189370225301|
|  Bruno| 0.5096018842446481|
| Carlos| 0.8325259388871524|
|  Diana|0.26322809041172357|
|Eduardo| 0.6702867696264135|
+-------+-------------------+
only showing top 5 rows

+-------+-------------------+------------------+-----------------+
|   nome|      aleatorio_0_1|   aleatorio_1_100|aleatorio_inteiro|
+-------+-------------------+------------------+-----------------+
|  Alice|0.15795279750951363|15.795279750951362|               15|
|  Bruno|  0.648787283930924|  64.8787283930924|               64|
| Carlos| 0.9529333503403405| 95.29333503403406|               95|
|  Diana| 0.31083

## 6. Amostragem de Dados

No PySpark, podemos extrair **amostras de um DataFrame** usando o método `sample()`. Isso é útil para análise exploratória, testes e simulações em datasets grandes, sem precisar processar todos os dados. Vamos analisar os exemplos:

* Amostragem **sem reposição**

  * `withReplacement=False` → cada registro só pode aparecer **uma vez** na amostra.
  * `fraction=0.2` → amostra corresponde a 20% do DataFrame original.
  * `seed=42` → garante **reprodutibilidade**, ou seja, sempre gera a mesma amostra.
  * **Uso típico:** análise exploratória quando não queremos duplicatas.
* Amostragem **com reposição**
  * `withReplacement=True` → cada registro pode aparecer **mais de uma vez** na amostra.
  * `fraction=0.3` → tamanho esperado da amostra é 30% do DataFrame original.
  * `seed=42` → garante que a amostra seja **reproduzível**.
  * **Uso típico:** simulações estatísticas, bootstrapping e testes que permitem repetição de registros.
* Sempre use `seed` para resultados consistentes.
* O `fraction` controla a **proporção da amostra** em relação ao dataset original.




In [8]:
df_grande = spark.range(0, 100).withColumn('valor', F.col('id') * 2)
df_grande.show(10)

amostra_sem_reposicao = df_grande.sample(withReplacement=False, fraction=0.2, seed=42)
amostra_sem_reposicao.show(10)

amostra_com_reposicao = df_grande.sample(withReplacement=True, fraction=0.3, seed=42)
amostra_com_reposicao.show(10)

+---+-----+
| id|valor|
+---+-----+
|  0|    0|
|  1|    2|
|  2|    4|
|  3|    6|
|  4|    8|
|  5|   10|
|  6|   12|
|  7|   14|
|  8|   16|
|  9|   18|
+---+-----+
only showing top 10 rows

+---+-----+
| id|valor|
+---+-----+
|  7|   14|
| 16|   32|
| 18|   36|
| 26|   52|
| 36|   72|
| 38|   76|
| 48|   96|
| 59|  118|
| 65|  130|
| 67|  134|
+---+-----+
only showing top 10 rows

+---+-----+
| id|valor|
+---+-----+
|  5|   10|
|  6|   12|
| 13|   26|
| 16|   32|
| 20|   40|
| 21|   42|
| 21|   42|
| 22|   44|
| 28|   56|
| 29|   58|
+---+-----+
only showing top 10 rows



## 7. Estatísticas por Grupos

In [12]:
# Dados expandidos para análise por grupos
dados_expandidos = [
    ("Alice", 25, "São Paulo", 5000.0),
    ("Bruno", 30, "São Paulo", 6500.0),
    ("Carlos", 28, "Rio de Janeiro", 4800.0),
    ("Diana", 35, "Rio de Janeiro", 7200.0),
    ("Eduardo", 32, "São Paulo", 5500.0),
    ("Fernanda", 29, "Rio de Janeiro", 6800.0),
    ("Gabriel", 27, "São Paulo", 5300.0),
    ("Helena", 31, "Rio de Janeiro", 6000.0),
]

# Criando o DataFrame usando o schema completo (StructType)
df_grupos = spark.createDataFrame(dados_expandidos, schema)

# Estatísticas por cidade
estatisticas_por_cidade = df_grupos.groupBy('cidade') \
    .agg(
        F.count('nome').alias('total_pessoas'),
        F.avg('idade').alias('idade_media'),
        F.avg('salario').alias('salario_medio'),
        F.min('salario').alias('salario_minimo'),
        F.max('salario').alias('salario_maximo'),
        F.stddev('salario').alias('desvio_padrao_salario')
    )

# Exibindo os resultados
estatisticas_por_cidade.show(truncate=False)


+--------------+-------------+-----------+-------------+--------------+--------------+---------------------+
|cidade        |total_pessoas|idade_media|salario_medio|salario_minimo|salario_maximo|desvio_padrao_salario|
+--------------+-------------+-----------+-------------+--------------+--------------+---------------------+
|São Paulo     |4            |28.5       |5575.0       |5000.0        |6500.0        |650.0                |
|Rio de Janeiro|4            |30.75      |6200.0       |4800.0        |7200.0        |1058.3005244258363   |
+--------------+-------------+-----------+-------------+--------------+--------------+---------------------+



## 8. Detecção de Outliers

In [10]:
quartis = df.select(
    F.expr('percentile_approx(salario, 0.25)').alias('q1'),
    F.expr('percentile_approx(salario, 0.75)').alias('q3')
).collect()[0]

q1 = quartis['q1']
q3 = quartis['q3']
iqr = q3 - q1
limite_inferior = q1 - 1.5 * iqr
limite_superior = q3 + 1.5 * iqr

outliers = df.filter((F.col('salario') < limite_inferior) | (F.col('salario') > limite_superior))
outliers.show()

+----+-----+------+-------+
|nome|idade|cidade|salario|
+----+-----+------+-------+
+----+-----+------+-------+



## 9. Correlação Entre Variáveis

Correlação é uma medida estatística que indica a **força e a direção** da relação linear entre duas variáveis numéricas. Em PySpark, podemos calcular a correlação usando o método `stat.corr()`.

* O valor retornado é um número entre **-1 e 1**:

  * `1` → correlação positiva perfeita (quando uma variável aumenta, a outra também aumenta proporcionalmente)
  * `-1` → correlação negativa perfeita (quando uma variável aumenta, a outra diminui proporcionalmente)
  * `0` → sem correlação linear aparente
* **Uso típico:** identificar relações lineares entre variáveis antes de análises exploratórias ou modelagem preditiva.

---

💡 **Interpretação prática:**

| Valor da correlação | Interpretação                   |
| ------------------- | ------------------------------- |
| 0.7 a 1.0           | Forte correlação positiva       |
| 0.3 a 0.7           | Correlação positiva moderada    |
| -0.3 a 0.3          | Correlação fraca ou inexistente |
| -0.7 a -0.3         | Correlação negativa moderada    |
| -1.0 a -0.7         | Forte correlação negativa       |

* **correlação não implica causalidade**. Valores altos indicam associação, mas não garantem relação de causa e efeito.

In [13]:
correlacao = df.stat.corr('idade', 'salario')
correlacao

0.06722971098151054

## 10. Finalizando Sessão Spark

In [ ]:
# spark.stop()  # Descomente para finalizar a sessão